<a href="https://colab.research.google.com/github/Witcape/Emotion_Recognition_in_Live_Video_Data/blob/main/Emotion_Recognition_in_Video_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### STEP 1 - Import all the Libraries and google drive

In [1]:
from sympy import *
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import sklearn
import tensorflow as tf
import pandas as pd
import sklearn
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.losses import SparseCategoricalCrossentropy

from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.activations import linear,relu,sigmoid
import cv2
import os
import PIL
import pathlib
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_datasets as tfds

In [2]:

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Step 2 - Loading Data & Preprocessing

In [2]:
train_data_dir = '/content/gdrive/MyDrive/Human emotions/train'
test_data_dir = '/content/gdrive/MyDrive/Human emotions/test'

In [3]:
picture_size = 48
batch_size = 32
no_of_classes = 7
epochs = 20

datagen_train = ImageDataGenerator (
                                    rescale=1./255,
                                    validation_split=0.2
                                    )

datagen_val   = ImageDataGenerator (
                                    rescale=1./255
                                    )
datagen_test   = ImageDataGenerator (
                                    rescale=1./255
                                    )

In [4]:
train_set = datagen_train.flow_from_directory(
    train_data_dir,
    target_size=(picture_size, picture_size),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

val_set = datagen_val.flow_from_directory(
    test_data_dir,
    target_size=(picture_size, picture_size),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)



FileNotFoundError: ignored

## Step 3 - Model

In [5]:
model = Sequential(
    [
        Conv2D(128, (3,3), 1,  activation = 'relu', padding='same', input_shape = (picture_size, picture_size, 1)),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        BatchNormalization(),

        Conv2D(256, (3,3), 1,  activation = 'relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(),
        Dropout(0.25),

        Conv2D(512, (3,3), 1,  activation = 'relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(),
        Dropout(0.25),

        Conv2D(256, (3,3), 1,  activation = 'relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(),
        Dropout(0.25),

        Conv2D(128, (3,3), 1,  activation = 'relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(),
        Dropout(0.25),

        Flatten(),

        Dense(512, activation = 'relu'),
        Dense(7, activation = 'softmax')
    ]
)

In [19]:
model.compile(
    optimizer = Adam(learning_rate = 0.01),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [20]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 48, 48, 128)       1280      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 24, 24, 128)      0         
 g2D)                                                            
                                                                 
 dropout_10 (Dropout)        (None, 24, 24, 128)       0         
                                                                 
 batch_normalization_10 (Bat  (None, 24, 24, 128)      512       
 chNormalization)                                                
                                                                 
 conv2d_11 (Conv2D)          (None, 24, 24, 256)       295168    
                                                                 
 batch_normalization_11 (Bat  (None, 24, 24, 256)     

In [21]:
log_dir = '/content/gdrive/MyDrive/Human emotions/Logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)

In [22]:
early_stopping = EarlyStopping(patience=5, monitor='val_loss', restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(factor=0.1, patience=3, monitor='val_loss', min_lr=0.00001)

In [ ]:
history = model.fit(
    train_set,
    epochs=2,
    validation_data=val_set,
    callbacks=[early_stopping, reduce_lr]
)

KeyboardInterrupt: ignored

In [8]:
loaded_model = load_model('/content/gdrive/MyDrive/Human emotions/val_acc_61', compile=False)

loaded_model.compile(optimizer=Adam(learning_rate=0.01), loss=SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

OSError: ignored

In [9]:
predictions = loaded_model.predict(val_set)

NameError: ignored

In [39]:
predicted_classes = np.argmax(predictions, axis=1)

In [ ]:
test_labels = np.concatenate([y for x, y in test], axis=0)

In [ ]:
correct_predictions = np.equal(predicted_classes, test_labels)

In [ ]:
accuracy = np.mean(correct_predictions)
print("Test Accuracy:", accuracy)

In [41]:
# Evaluate the model on the test dataset with the specified batch size
evaluation = loaded_model.evaluate(val_set, batch_size=len(val_set))

# The evaluate method returns the loss and all the metrics specified during compilation.
accuracy = evaluation[1]

print("Test Accuracy:", accuracy)


InvalidArgumentError: ignored

## Trying to make it work with video frames